# Lecture 01 — Introduction to Data Mining (Python)

**Term:** Fall 2025  

**Week/Topic:** Lecture 01 — Data Exploration & Linear Regression  

**Instructor:** Dr. Bushaj  

### What we'll cover
- Colab/Drive setup & data-path conventions
- Imports & aliases (`numpy`→`np`, `pandas`→`pd`, `matplotlib.pyplot`→`plt`, `sklearn`, `seaborn`, `dmba`)
- Load dataset: **West Roxbury Housing**
- Clean column names & access patterns (`loc`, `iloc`, dot notation)
- Quick EDA: shape, head, dtypes, describe, value counts
- Sampling: random, weighted oversampling, group-wise sampling
- Data quality checks: missing values & "unseen" missing values (e.g., `YR_BUILT == 0`)
- Handling missing data & types (impute, fill, convert to categorical)
- Visualization: distributions & correlation heatmap
- Scaling/normalizing features (StandardScaler, MinMaxScaler)
- Partitioning data (train/validation/test, stratified option)
- Linear Regression: fit, predict, coefficients
- Model evaluation: MSE, RMSE, R², residuals; `regressionSummary`
- Scoring validation data & predicting for new records

---

> We often import packages with short aliases for readability:
>
> ```python
> import <package> as <alias>
> ```
> Common aliases include `np` (NumPy), `pd` (pandas), and `plt` (matplotlib.pyplot).
>
> This notebook expects a data path appropriate to your environment (e.g., a Drive-mounted path in Colab).


## Import Needed Functionality

In [4]:
# Import necessary libraries for data manipulation, modeling, and plotting

# install dmba package
!pip install dmba
# import the utility function regressionSummary
from dmba import regressionSummary
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split   # For splitting the dataset
from sklearn.metrics import r2_score,mean_squared_error    # For evaluating the model's performance
from sklearn.linear_model import LinearRegression  # For performing linear regression# Linear Regression model
import matplotlib.pyplot as plt     # For plotting actual vs predicted values
#The abbreviations pd, np, and sm are commonly used in the data science community.
#help(sklearn)

## 1. Data Exploration in Python (Loading, View, and Summarize)

### Load Data

In [5]:
#create a folder with all the data files there and get the path to that folder
my_drive_path = "/content/drive/MyDrive/SUNY/Class Material/2024 Fall/MSA550A/Python Class Work/msa550-code-files/data/"

**Dataset: West Roxbury Housing Data**


    'TOTAL VALUE': 'Total assessed value of the property (in dollars)',
    'TAX': 'Annual property tax (in dollars)',
    'LOT SQFT': 'Lot size in square feet',
    'YR BUILT': 'Year the property was built',
    'GROSS AREA': 'Gross area of the property (in square feet)',
    'LIVING AREA': 'Living area in square feet',
    'FLOORS': 'Number of floors in the property',
    'ROOMS': 'Total number of rooms in the property',
    'BEDROOMS': 'Number of bedrooms in the property',
    'FULL BATH': 'Number of full bathrooms',
    'HALF BATH': 'Number of half bathrooms',
    'KITCHEN': 'Indicates if the property has a kitchen (1 = yes, 0 = no)',
    'FIREPLACE': 'Number of fireplaces in the property',
    'REMODEL': 'Indicates whether the property has been remodeled'

In [6]:
# Load data int a Pandas Dataframe
housing_df = pd.read_csv(my_drive_path+'WestRoxbury.csv')

# Show the shape (dimensions) of the DataFrame: (number of rows, number of columns)
housing_df.shape

# Display the first 5 rows to get an initial look at the dataset
housing_df.head()  #show the 1st five rows
#print(housing_df)  #show all the data

# In case you want to view all the rows at once, you can adjust the display option:
pd.set_option('display.max_rows', None)
#print(housing_df)


# Print the list of column names to see which variables are present in the dataset
print(housing_df.columns)  # print a list of variables
#it can be tricky to access these columns with spaces. We can edit all
housing_df['TOTAL VALUE '] #you need to add those spaces. The spaces also affect this housing_df.TOTAL VALUE --wont work

# Rename columns: replace spaces with '_' housing_df = housing_df.rename(columns={'TOTAL VALUE ': 'TOTAL_VALUE'})
#housing_df = housing_df.rename(columns={'TOTAL VALUE ': 'TOTAL_VALUE'})
# this will change all the names of the columns according to this rule
housing_df.columns = [s.strip().replace(' ', '_')  for s in housing_df.columns]

# Print the cleaned column names to verify the changes
print(housing_df.columns)


# Accessing subsets of the data
# Pandas provides two main methods to access rows in a DataFrame: `loc` and `iloc`.
# `loc`: Access rows by label (name or index)
# `iloc`: Access rows by integer positions (numerical index)
# Example: Let's show the first four rows using both methods.

# Using `loc` to show rows 0 to 3 (inclusive)
housing_df.loc[0:3]

# Using `iloc` to show rows 0 to 3 (iloc excludes the last index, so 4 is not included)
housing_df.iloc[0:4]


# There are different ways to access the first 10 values in a column (TOTAL_VALUE)

# Option 1: Using the column name directly and `iloc` to limit rows
housing_df['TOTAL_VALUE'].iloc[0:10]

# Option 2: Using `iloc` first to slice the rows, then selecting the column
housing_df.iloc[0:10]['TOTAL_VALUE']

# Option 3: Using dot notation (only works if the column name has no spaces)
housing_df.iloc[0:10].TOTAL_VALUE


# Accessing specific rows and columns
# Show the fifth row of the first 10 columns in different ways:
housing_df.iloc[4][0:10]  # Method 1: returning a single row as a Series
housing_df.iloc[4, 0:10]  # Method 2: similar but simpler syntax
housing_df.iloc[4:5, 0:10]  # Method 3: using a slice returns a DataFrame (useful if you need a DataFrame, not Series)

# Concatenating multiple columns (including non-consecutive columns)
# Using pd.concat to combine two separate slices of columns (axis=1 means we combine them side by side as columns)
pd.concat([housing_df.iloc[4:6, 0:2], housing_df.iloc[4:6, 4:6]], axis=1)


# Accessing a full column in various ways
housing_df.iloc[:, 0:1]  # Access first column using iloc
housing_df.TOTAL_VALUE  # Dot notation to access TOTAL_VALUE column
housing_df['TOTAL_VALUE'][0:10]  # Access the first 10 rows of the TOTAL_VALUE column

# Basic descriptive statistics

# Find and print the number of rows (length) of the TOTAL_VALUE column
print('Number of rows:', len(housing_df['TOTAL_VALUE']))
# Calculate and print the mean of the TOTAL_VALUE column
print('Mean of TOTAL_VALUE:', housing_df['TOTAL_VALUE'].mean())

# Display a summary of descriptive statistics for all columns (mean, median, std, etc.)
housing_df.describe()

Index(['TOTAL VALUE ', 'TAX', 'LOT SQFT ', 'YR BUILT', 'GROSS AREA ',
       'LIVING AREA', 'FLOORS ', 'ROOMS', 'BEDROOMS ', 'FULL BATH',
       'HALF BATH', 'KITCHEN', 'FIREPLACE', 'REMODEL'],
      dtype='object')
Index(['TOTAL_VALUE', 'TAX', 'LOT_SQFT', 'YR_BUILT', 'GROSS_AREA',
       'LIVING_AREA', 'FLOORS', 'ROOMS', 'BEDROOMS', 'FULL_BATH', 'HALF_BATH',
       'KITCHEN', 'FIREPLACE', 'REMODEL'],
      dtype='object')
Number of rows: 5802
Mean of TOTAL_VALUE: 392.6857149258877


,TOTAL_VALUE,TAX,LOT_SQFT,YR_BUILT,GROSS_AREA,LIVING_AREA,FLOORS,ROOMS,BEDROOMS,FULL_BATH,HALF_BATH,KITCHEN,FIREPLACE
count,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.000000,5802.00000,5802.000000
mean,392.685715,4939.485867,6278.083764,1936.744916,2924.842123,1657.065322,1.683730,6.994829,3.230093,1.296794,0.613926,1.01534,0.739917
std,99.177414,1247.649118,2669.707974,35.989910,883.984726,540.456726,0.444884,1.437657,0.846607,0.522040,0.533839,0.12291,0.565108
min,105.000000,1320.000000,997.000000,0.000000,821.000000,504.000000,1.000000,3.000000,1.000000,1.000000,0.000000,1.00000,0.000000
25%,325.125000,4089.500000,4772.000000,1920.000000,2347.000000,1308.000000,1.000000,6.000000,3.000000,1.000000,0.000000,1.00000,0.000000
50%,375.900000,4728.000000,5683.000000,1935.000000,2700.000000,1548.500000,2.000000,7.000000,3.000000,1.000000,1.000000,1.00000,1.000000
75%,438.775000,5519.500000,7022.250000,1955.000000,3239.000000,1873.750000,2.000000,8.000000,4.000000,2.000000,1.000000,1.00000,1.000000
max,1217.800000,15319.000000,46411.000000,2011.000000,8154.000000,5289.000000,3.000000,14.000000,9.000000,5.000000,3.000000,2.00000,4.000000
